### download dahiti lake height in the hma

In [ ]:
import os
import json
import requests
import pprint

In [ ]:
dahiti_key = '5C5F7D96A01C2A3C2D0F11636BF2F96C1113A22989086EEDCBC1E02710E7474D'
dir_download = 'data/altimetry-water-level/dahiti'


#### download

In [ ]:
''' Download path were all water level time series will be stored '''
if not os.path.isdir(dir_download):
	os.mkdir(dir_download)
API_KEY = dahiti_key
OUTPUT_FORMAT = 'netcdf' ## Option: ascci, json, netcdf, csv 

url = 'https://dahiti.dgfi.tum.de/api/v2/list-targets/'
args = {}
''' User configuration '''
args['api_key'] = API_KEY
''' Search options '''
# args['basin'] = 'Amazon'
# args['continent'] = 'Asia'
#args['country'] = 'de'
args['min_lon'], args['max_lon'] = 100, 106   #### 66-106
args['min_lat'], args['max_lat'] = 26, 45    #### 26-45


In [7]:
''' send request as method POST '''
response = requests.post(url, data=args)
if response.status_code == 200:
	''' convert json string in python list '''
	targets = json.loads(response.text)['data']
	print ('Dataset(s) found:',len(targets))		
	for target in targets:
		# print (target)		
		path_target_file = dir_download + '/' + str(target['dahiti_id']) + '.nc'
		if os.path.exists(path_target_file):
			print(f"File {target['dahiti_id']} already exists. Skipping download.")
			continue

		''' download water level time series '''
		url = 'https://dahiti.dgfi.tum.de/api/v2/download-water-level/'
		args = {}
		args['api_key'] = API_KEY
		args['dahiti_id'] = target['dahiti_id']
		args['format'] = OUTPUT_FORMAT		
		path_output = os.path.abspath(dir_download+'/'+str(target['dahiti_id'])+'.nc')
		print ('Downloading ... ',target['dahiti_id'],'->',target['target_name'].encode("utf8"),'('+path_output+')')		
		response_download = requests.post(url, json=args)
		if response_download.status_code == 200:
			with open(path_output, 'wb') as f:
				for chunk in response_download.iter_content(chunk_size=1024): 
					if chunk:
						f.write(chunk)
		else:
			print ('Error: `download-water-level` request failed!')
			data = json.loads(response_download.text)
			pprint.pprint(data)
			# sys.exit(0)
else:
	print ('Error: `list-targets` request failed!')
	data = json.loads(response.text)
	pprint.pprint(data)



Dataset(s) found: 17
